In [92]:
import scipy as sp
import numpy as np
import scipy.sparse.linalg
import scipy.sparse
import networkx as nx
from networkx.algorithms.link_analysis.pagerank_alg import google_matrix
from sklearn.preprocessing import normalize
from random import sample
import random
import operator


In [93]:
G = nx.read_edgelist('./Networks/p2p-Gnutella25.txt', comments='#',
                     create_using=nx.DiGraph(), 
                     delimiter='\t', 
                     nodetype=int, 
                     encoding='utf-8')


# Creating the base set

We create a function that constructs a random set of 2000 nodes, as a root set, and then extends it to create the base set for our algorithms.
The extension is based on the edges and the connection of two nodes.
If the base set nodes are not connected, we keep the largest connected component

In [94]:
def nodeSet(G):
    '''
    A function that creates a random set of 2000 nodes, as a root set, and then extends it to create the base set for our algorithms.
    If the base set nodes are not connected, we keep the largest connected component
    :param G: The initial graph.
    :returns H: The base set of nodes, from our initial graph.
    '''
    sampleN = sample(G.nodes(), 2000)
    edges=list(G.out_edges(sampleN))
    in_edges=G.in_edges(sampleN)
    edges.extend(in_edges)
    edgesG=nx.DiGraph(edges)
    newSampleNodes=edgesG.nodes()
    H=G.subgraph(newSampleNodes)
    if nx.is_connected(nx.Graph(H))==False:
        largest_cc = max(nx.connected_components(nx.Graph(H)), key=len)
        H=H.subgraph(largest_cc)
    
    return H

In [95]:
graph=nodeSet(G)
print(len(graph.nodes()))

8041


# Salsa algorithm

The SALSA algorithm (Stochastic Approach for Link-Structure Analysis) is an algorithm inspired by PageRank and HITS. More precisely SALSA is a modification of HITS algorithm, that conducts a random walk (an attribute of PageRank) between two graphs. The two graphs are created by taking the graph g and looking at it as a bipartite graph. On the one side the nodes represent hubs and on the other side authorities. Here we need max_iter==600 to converge.

In [96]:
def salsa(G, max_iter=600, tol=1.0e-8, nstart=None, normalized=True):
    '''
    SALSA implementation.
    :param G: the Graph G that we want to find the SALSA hits and authorities.
    :param max_iter: maximum number of iterations before the algorithm needs to converge. Else break.
    :param tol: tolerance of permited error til convergence.
    '''
    A = nx.adjacency_matrix(G, nodelist=list(G)).astype('float')
    (rows, cols) = A.shape 
    colNormA=normalize(A, norm='l1', axis=0)
    rowNormA=normalize(A, norm='l1', axis=1)
    
    authM = rowNormA.T * colNormA 
    a = np.ones((rows, 1)) / rows  # authority vector
    
    i = 0
    while True:
        flag = a
        a = authM * a
        a = a / a.max()
        error = np.absolute(a - flag).sum()
        if error < tol:
            break
        if i > max_iter:
            print("BREAK")
            print(i)
            break
            #raise nx.PowerIterationFailedConvergence(max_iter) --> Doesn't work, apparently....
        i += 1
    a = np.asarray(a).flatten()
    h = np.asarray(colNormA * a).flatten()  # h=adjacency_matrix_c*a
    if normalized:
        #print(dict(G.in_degree()))
        h = h / sum(dict(G.in_degree()).values())
        a = a / sum(dict(G.out_degree()).values())
    
    hubs = dict(zip(G, map(float, h)))
    authorities = dict(zip(G, map(float, a)))
    
    return hubs, authorities
        
    

# Checking the overlaps 

We create a function that checks the overlaps between the needed set of hubs, authorities and PageRank scores.

In [97]:
'''def checkOverlaps(sA,sH,hA,hH,pr):
    overlapSASH=[]
    overlapHAHH=[]
    overlapPRHA=[]
    overlapPRSA=[]
    overlapHASA=[]
    overlapSHHH=[]
    
    
    for i in range(1,21):
        c1=c2=c3=c4=c5=c6=0
        for j in range(1,i):
            if list(sA.keys())[j]==list(sH.keys())[j]:
                c1 += 1
            if list(hA.keys())[j]==list(hH.keys())[j]:
                c2 += 1
            if list(pr.keys())[j]==list(hA.keys())[j]:
                c3 += 1
            if list(pr.keys())[j]==list(sA.keys())[j]:
                c4 += 1
            if list(hA.keys())[j]==list(sA.keys())[j]:
                c5 += 1
            if list(sH.keys())[j]==list(hH.keys())[j]:
                c6 += 1
        

        
        overlapSASH.append(c1)
        overlapHAHH.append(c2)
        overlapPRHA.append(c3)
        overlapPRSA.append(c4)
        overlapHASA.append(c5)
        overlapSHHH.append(c6)
        

    
    return sum(overlapSASH)/20, sum(overlapHAHH)/20, sum(overlapPRHA)/20, sum(overlapPRSA)/20, sum(overlapHASA)/20, sum(overlapSHHH)/20
        
        

        '''

'def checkOverlaps(sA,sH,hA,hH,pr):\n    overlapSASH=[]\n    overlapHAHH=[]\n    overlapPRHA=[]\n    overlapPRSA=[]\n    overlapHASA=[]\n    overlapSHHH=[]\n    \n    \n    for i in range(1,21):\n        c1=c2=c3=c4=c5=c6=0\n        for j in range(1,i):\n            if list(sA.keys())[j]==list(sH.keys())[j]:\n                c1 += 1\n            if list(hA.keys())[j]==list(hH.keys())[j]:\n                c2 += 1\n            if list(pr.keys())[j]==list(hA.keys())[j]:\n                c3 += 1\n            if list(pr.keys())[j]==list(sA.keys())[j]:\n                c4 += 1\n            if list(hA.keys())[j]==list(sA.keys())[j]:\n                c5 += 1\n            if list(sH.keys())[j]==list(hH.keys())[j]:\n                c6 += 1\n        \n\n        \n        overlapSASH.append(c1)\n        overlapHAHH.append(c2)\n        overlapPRHA.append(c3)\n        overlapPRSA.append(c4)\n        overlapHASA.append(c5)\n        overlapSHHH.append(c6)\n        \n\n    \n    return sum(overlapSASH)

In [98]:
def checkOverlaps(sA,sH,hA,hH,pr):
    '''
    Checks overlaps by creating two sets from lists, repetitively, and checking the intersection.
    :param sA: SALSA Authority scores
    :param sH: SALSA Hub scores
    :param hA: HITS Authority scores
    :param hH: HITS Hub scores 
    :param pr: PageRank scores
    :returns: The average of the percentages of each overlap
    '''
    overlapSASH=[]
    overlapHAHH=[]
    overlapPRHA=[]
    overlapPRSA=[]
    overlapHASA=[]
    overlapSHHH=[]
    
    
    for i in range(1,21):
        overlapSASH.append((len(list(set(list(sA.keys())[:i])&set(list(sH.keys())[:i]))))/i)
        overlapHAHH.append((len(list(set(list(hA.keys())[:i])&set(list(hH.keys())[:i]))))/i)
        overlapPRHA.append((len(list(set(list(pr.keys())[:i])&set(list(hA.keys())[:i]))))/i)
        overlapPRSA.append((len(list(set(list(pr.keys())[:i])&set(list(sA.keys())[:i]))))/i)
        overlapHASA.append((len(list(set(list(hA.keys())[:i])&set(list(sA.keys())[:i]))))/i)
        overlapSHHH.append((len(list(set(list(sH.keys())[:i])&set(list(hH.keys())[:i]))))/i)
        

    
    return sum(overlapSASH)/20, sum(overlapHAHH)/20, sum(overlapPRHA)/20, sum(overlapPRSA)/20, sum(overlapHASA)/20, sum(overlapSHHH)/20
        
        

        

In [99]:
overlapSASHT=[]
overlapHAHHT=[]
overlapPRHAT=[]
overlapPRSAT=[]
overlapHASAT=[]
overlapSHHHT=[]
    

for i in range(10):
    
    graph=nodeSet(G)
    
    ######SALSA########
    h,a=salsa(graph)
    sorted_tuples=(sorted(a.items(), key=operator.itemgetter(1),reverse=True))
    a = {k: v for k, v in sorted_tuples}
    sorted_tuples=(sorted(h.items(), key=operator.itemgetter(1),reverse=True))
    h = {k: v for k, v in sorted_tuples}    
    
    ######HITS#########
    
    hits_h,hits_a=nx.hits(graph,max_iter=600)
    sorted_tuples=(sorted(hits_a.items(), key=operator.itemgetter(1),reverse=True))
    hits_a = {k: v for k, v in sorted_tuples}
    sorted_tuples=(sorted(hits_h.items(), key=operator.itemgetter(1),reverse=True))
    hits_h = {k: v for k, v in sorted_tuples}
    
    #######PAGERANK#######
    
    pr = nx.pagerank(graph, alpha=0.9,max_iter=600)
    sorted_tuples=(sorted(pr.items(), key=operator.itemgetter(1),reverse=True))
    pr = {k: v for k, v in sorted_tuples}

    #######OVERLAP########
    print(i)
    print(list(a.keys())[:20])
    print(list(h.keys())[:20])
    print(list(hits_a.keys())[:20])
    print(list(hits_h.keys())[:20])
    print(list(pr.keys())[:20])
    
    
    overlapSASH, overlapHAHH, overlapPRHA, overlapPRSA, overlapHASA, overlapSHHH=checkOverlaps(a,h,hits_a,hits_h,pr)
    
    overlapSASHT.append(overlapSASH)
    overlapHAHHT.append(overlapHAHH)
    overlapPRHAT.append(overlapPRHA)
    overlapPRSAT.append(overlapPRSA)
    overlapHASAT.append(overlapHASA)
    overlapSHHHT.append(overlapSHHH)

0
[392, 1941, 464, 1349, 2051, 797, 1009, 685, 97, 1183, 10836, 12221, 4920, 3491, 4766, 1028, 2237, 2306, 6803, 9765]
[16067, 1397, 17679, 7210, 797, 5003, 2653, 20245, 1225, 12204, 9837, 12397, 13154, 21899, 16683, 2115, 22548, 3808, 20267, 478]
[2492, 2494, 1183, 4473, 415, 4475, 820, 431, 2397, 3689, 82, 2495, 9470, 6513, 17842, 15461, 3872, 2058, 1941, 10163]
[17652, 4115, 10561, 7768, 771, 14209, 19814, 8591, 13414, 1705, 1370, 16128, 9442, 7178, 13296, 4840, 13091, 13166, 7628, 5489]
[797, 464, 2051, 1349, 1941, 8770, 10836, 12221, 6009, 392, 4939, 5692, 3491, 1673, 97, 685, 2545, 4907, 15895, 1387]
1
[685, 1183, 464, 10836, 392, 97, 4730, 1941, 254, 988, 6098, 296, 2051, 4920, 1349, 11878, 1009, 11379, 3491, 4766]
[1397, 16067, 15816, 2115, 5003, 2653, 9837, 20245, 1225, 17679, 2291, 12397, 13154, 7210, 1463, 21490, 16992, 1257, 20267, 12204]
[6715, 2527, 1755, 12125, 4156, 9663, 12597, 12877, 7251, 9, 3854, 16217, 10669, 15714, 3212, 4708, 16219, 13648, 16208, 10541]
[1397, 15

In [100]:
print("Overlaps between SALSA authorities and SALSA Hubs:  ",overlapSASHT)
print("The average:  ",sum(overlapSASHT)/10)
print()
print("Overlaps between HITS authorities and HITS Hubs:  ",overlapHAHHT)
print("The average:  ",sum(overlapHAHHT)/10)
print()
print("Overlaps between PageRank and HITS Authorities:  ",overlapPRHAT)
print("The average:  ",sum(overlapPRHAT)/10)
print()
print("Overlaps between PageRank and SALSA Authorities:  ",overlapPRSAT)
print("The average:  ",sum(overlapPRSAT)/10)
print()
print("Overlaps between SALSA authorities and HITS Authorities:  ",overlapHASAT)
print("The average:  ",sum(overlapHASAT)/10)
print()
print("Overlaps between SALSA Hubs and HITS Hubs:  ",overlapSHHHT)
print("The average:  ",sum(overlapSHHHT)/10)

Overlaps between SALSA authorities and SALSA Hubs:   [0.06572031619051744, 0.0, 0.0, 0.0079093567251462, 0.04399412571432696, 0.0, 0.02244044547643619, 0.0, 0.0, 0.06572031619051744]
The average:   0.020578456029694422

Overlaps between HITS authorities and HITS Hubs:   [0.0, 0.020880295100496337, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
The average:   0.0020880295100496335

Overlaps between PageRank and HITS Authorities:   [0.005131578947368421, 0.0, 0.0, 0.0, 0.0, 0.0, 0.017308866529067764, 0.0, 0.1798869828571841, 0.0]
The average:   0.020232742833362027

Overlaps between PageRank and SALSA Authorities:   [0.563203198375412, 0.41756252497932056, 0.2614856696880691, 0.4524399237673231, 0.5355669508939632, 0.4692320257142238, 0.4474303010371122, 0.45351910194208944, 0.5483794061116043, 0.5846939043069074]
The average:   0.4733513006816025

Overlaps between SALSA authorities and HITS Authorities:   [0.043570149106139815, 0.0, 0.0, 0.1279639649995687, 0.0, 0.08211343180383428, 0.14759745